<a href="https://colab.research.google.com/github/madhvirathod/Bike-Sharing/blob/main/Exercise%202.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keplergl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922317 sha256=6320faf3b3c0ddd3683840bc780e9a9ca8033d30a8bccfbc7a85170b5a8b4922
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl


In [2]:
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NewYork_data_clean.csv')

<ipython-input-4-ec70a791df5b>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NewYork_data_clean.csv')


In [5]:
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,0.0,344EB2CD04CD1B71,classic_bike,46:17.9,54:21.7,Lafayette St & Jersey St,5561.06,W 15 St & 6 Ave,5989.02,40.724561,-73.995653,40.738046,-73.996430,member,4/13/2022,15.4,both
1,1.0,17248543C251F4FB,electric_bike,08:25.3,23:26.8,W 29 St & 9 Ave,6416.06,Spring St & Hudson St,5653.12,40.750073,-73.998393,40.725840,-74.007653,casual,6/24/2022,21.9,both
2,2.0,43658D9AB9938656,classic_bike,39:18.9,09:58.4,N 11 St & Kent Ave,5489.04,Kent Ave & N 7 St,5489.03,40.722482,-73.959219,40.720368,-73.961651,casual,5/28/2022,20.4,both
3,3.0,0158721C43867EFE,classic_bike,33:55.1,40:16.9,DeKalb Ave & Hudson Ave,4513.06,Flushing Ave & Vanderbilt Ave,4762.05,40.689888,-73.981013,40.697950,-73.970776,casual,4/29/2022,10.4,both
4,4.0,B08E1E3FFC1F965B,classic_bike,43:40.1,58:17.8,Christopher St & Greenwich St,5847.01,7 Ave S & Bleecker St,5805.07,40.732916,-74.007114,40.732139,-74.003638,member,9/30/2022,14.5,both


In [6]:
df.columns

Index(['Unnamed: 0', 'ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgTemp', '_merge'],
      dtype='object')

In [7]:
df.shape

(801824, 17)

In [8]:
df = df.drop(columns=["Unnamed: 0"])

In [9]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgTemp', '_merge'],
      dtype='object')

In [10]:
df.drop(columns = {'_merge'}, inplace = True)

In [11]:
# Create a value column and group by start and end station
df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()

In [12]:
df_group

,start_station_name,end_station_name,value
0,1 Ave & E 110 St,1 Ave & E 110 St,14
1,1 Ave & E 110 St,1 Ave & E 18 St,1
2,1 Ave & E 110 St,1 Ave & E 30 St,1
3,1 Ave & E 110 St,1 Ave & E 94 St,5
4,1 Ave & E 110 St,2 Ave & E 104 St,9
...,...,...,...
261311,Yankee Ferry Terminal,Pioneer St & Richards St,1
261312,Yankee Ferry Terminal,Pioneer St & Van Brunt St,1
261313,Yankee Ferry Terminal,Soissons Landing,122
261314,Yankee Ferry Terminal,South St & Whitehall St,5


In [13]:
print(df_group['value'].sum())
print(df.shape)

799824
(801824, 16)


In [14]:
df_group['value'].describe()

,value
count,261316.000000
mean,3.060754
std,4.853766
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,331.000000


In [15]:
df_group = df_group.rename(columns={'value': 'trips'})

In [16]:
df_group.head()

,start_station_name,end_station_name,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,14
1,1 Ave & E 110 St,1 Ave & E 18 St,1
2,1 Ave & E 110 St,1 Ave & E 30 St,1
3,1 Ave & E 110 St,1 Ave & E 94 St,5
4,1 Ave & E 110 St,2 Ave & E 104 St,9


In [17]:
subset_df = df[['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

In [18]:
merged_df = pd.merge(subset_df, df_group, on=['start_station_name', 'end_station_name'], how='inner')

In [19]:
final_df = merged_df.drop_duplicates(subset=['start_station_name'])

In [20]:
final_df.head()

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,trips
0,Lafayette St & Jersey St,W 15 St & 6 Ave,40.724561,-73.995653,40.738046,-73.996430,9
9,W 29 St & 9 Ave,Spring St & Hudson St,40.750073,-73.998393,40.725840,-74.007653,6
15,N 11 St & Kent Ave,Kent Ave & N 7 St,40.722482,-73.959219,40.720368,-73.961651,46
61,DeKalb Ave & Hudson Ave,Flushing Ave & Vanderbilt Ave,40.689888,-73.981013,40.697950,-73.970776,8
69,Christopher St & Greenwich St,7 Ave S & Bleecker St,40.732916,-74.007114,40.732139,-74.003638,7


In [21]:
final_df.to_csv('final_df_locations_for_map.csv')

In [22]:
# Create KeplerGl instance
m = KeplerGl(height = 700, data={"data_1": final_df})
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':                    start_station_name               end_station_name  \
0            …

In [23]:
m

KeplerGl(data={'data_1':                    start_station_name               end_station_name  \
0            …

In [24]:
from google.colab import output
output.enable_custom_widget_manager()

In [25]:
m = KeplerGl(height=700, data={"data_1": final_df})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [26]:
m

KeplerGl(data={'data_1':                    start_station_name               end_station_name  \
0            …

In [33]:
config = m.config

In [34]:
config

{'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['data_1'],
     'id': 'oc3qk94l9',
     'name': ['trips'],
     'type': 'range',
     'value': [3, 213],
     'enlarged': False,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None,
     'speed': 1}],
   'layers': [{'id': 'a3g38kf',
     'type': 'point',
     'config': {'dataId': 'data_1',
      'label': 'start',
      'color': [206, 64, 170],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'start_lat', 'lng': 'start_lng', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'str

In [35]:
import json
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

In [36]:
m.save_to_html(file_name="NewYork_Bike_Trip_map.html", config=config)

Map saved to NewYork_Bike_Trip_map.html!


In [37]:
from google.colab import files
files.download("NewYork_Bike_Trip_map.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>